In [1]:
!pip install spotipy
!pip install seaborn

In [2]:
import os
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict



In [3]:
data = pd.read_csv("C:/Users/tmend/OneDrive/Escritorio/Vanny/machine-learning/intership codealpha/week 1/data.csv")


In [4]:
data.describe()

,duration,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,danceability,repeated_plays
count,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,3.404900,0.587650,5.05000,-6.577120,0.560000,0.075461,0.314539,0.070682,0.161737,0.517354,121.548260,0.647900,0.490000
std,0.927022,0.168273,3.82542,2.447338,0.498888,0.068065,0.281076,0.221947,0.112657,0.237512,29.148613,0.126942,0.522136
min,0.730000,0.157000,0.00000,-14.067000,0.000000,0.025300,0.000280,0.000000,0.034100,0.059200,71.884000,0.352000,0.000000
25%,2.910000,0.477750,1.00000,-7.676250,0.000000,0.036100,0.090750,0.000000,0.088375,0.329000,97.476250,0.566750,0.000000
50%,3.370000,0.609000,5.00000,-6.262500,1.000000,0.051800,0.238500,0.000000,0.120000,0.542000,117.037500,0.663500,0.000000
75%,3.630000,0.707750,8.00000,-4.786750,1.000000,0.080650,0.519750,0.000083,0.205250,0.711500,141.733750,0.734000,1.000000
max,10.220000,0.883000,11.00000,-2.724000,1.000000,0.368000,0.941000,0.941000,0.509000,0.967000,180.917000,0.910000,2.000000


In [5]:
data.head(10)

,id,name,duration,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,danceability,repeated_plays,timestamp
0,4ZtFanR9U6ndgddUvNcjcG,Good 4 U Olivia Rodrigo,2.97,0.664,9,-5.044,1,0.1540,0.33500,0.000000,0.0849,0.688,166.928,0.563,1,08:00:00
1,5fxyZf6m2xHeSrOzUfcJrq,Stay The Kid LAROI & Justin Bieber,2.30,0.506,8,-11.275,1,0.0589,0.37900,0.868000,0.1100,0.454,170.054,0.564,0,08:00:00
2,5nujrmhLynf4yMoMtj8AQF,Levitating Dua Lipa feat. DaBaby,3.38,0.825,6,-3.787,0,0.0601,0.00883,0.000000,0.0674,0.915,102.977,0.702,1,09:20:00
3,4iJyoBOLtHqaGxP12qzhQI,Peaches Justin Bieber feat. Daniel Caesar & Gi...,3.30,0.696,0,-6.181,1,0.1190,0.32100,0.000000,0.4200,0.464,90.030,0.677,0,10:20:00
4,1SC5rEoYDGUK4NfG82494W,Montero (Call Me By Your Name) Lil Nas X,2.30,0.503,8,-6.725,0,0.2200,0.29300,0.000000,0.4050,0.710,178.781,0.593,1,11:20:00
5,3DarAbFujv6eYNliUTyqtz,Kiss Me More (feat. SZA) Doja Cat,3.48,0.705,8,-3.463,1,0.0284,0.25900,0.000089,0.1200,0.781,110.970,0.764,0,12:20:00
6,02VBYrHfVwfEWXk5DXyf0T,Leave The Door Open Silk Sonic (Bruno Mars & A...,4.03,0.616,5,-7.964,1,0.0324,0.18200,0.000000,0.0927,0.719,148.088,0.586,0,13:20:00
7,5wANPM4fQCJwkGd4rN57mH,drivers license Olivia Rodrigo,4.03,0.431,10,-8.810,1,0.0578,0.76800,0.000014,0.1060,0.137,143.875,0.561,0,15:20:00
8,0VjIjW4GlUZAMYd2vXMi3b,Blinding Lights The Weeknd,3.33,0.730,1,-5.934,1,0.0598,0.00146,0.000095,0.0897,0.334,171.005,0.514,1,16:20:00
9,6HU7h9RYOaPRFeh0R3UeAr,Deja Vu Olivia Rodrigo,3.59,0.612,2,-7.222,1,0.1120,0.58400,0.000006,0.3700,0.178,180.917,0.442,1,15:20:00


In [6]:
data.shape

(100, 16)

In [7]:
# Get unique categories and count members in each category
category_counts = {col: data[col].value_counts() for col in data.columns}
print(category_counts)
# Print results
for col, counts in category_counts.items():
    print(f"Column: {col}")
    print(counts, "\n")

{'id': id
5nujrmhLynf4yMoMtj8AQF    3
54bFM56PmE4YLRnqpW6Tha    3
2V2lFQQQR3iqSUG5jwDQd9    2
5wANPM4fQCJwkGd4rN57mH    2
6UelLqGlWMcVH1E5c4H7lY    2
                         ..
3fx31TjnwvwZIyAGHhOhA1    1
50nfwKoDiSYg8zOCREWAm5    1
4eAtgAE8EJ8tBQsAnZEonX    1
3Vi5XqYrmQgOYBajMWSvCi    1
5jsw9uXEGuKyJzs0boZ1bT    1
Name: count, Length: 86, dtype: int64, 'name': name
Therefore I Am Billie Eilish                3
Yonaguni Bad Bunny                          2
Montero (Call Me By Your Name) Lil Nas X    2
Kiss Me More (feat. SZA) Doja Cat           2
Hold On Justin Bieber                       2
                                           ..
Blinding Lights - 2019 The Weeknd           1
Need To Know Doja Cat                       1
Someone You Loved Lewis Capaldi             1
LA CANCIÓN J Balvin & Bad Bunny             1
Stitches Shawn Mendes                       1
Name: count, Length: 90, dtype: int64, 'duration': duration
4.03    5
3.38    4
3.30    4
3.59    4
3.37    4
       ..
3.13

In [8]:
# because the user id and song names are repeatitive, so it is important to include these two features


# Model 1 with LR
## repeated plays as target var, remove row when this == 2 because it is a problem of 2 classes
### Data preprocessing, normalized all features Z-score

In [9]:
data = data[data['repeated_plays'] != 2]

#target encoding for the song name
target_mean = data.groupby('name')['repeated_plays'].mean()
target_mean

name
All Too Well Taylor Swift                  0.0
All of Me John Legend                      0.0
Arcade Duncan Laurence                     0.0
Astronaut In The Ocean Masked Wolf         0.0
At My Worst (feat. Kehlani) Pink Sweat$    1.0
                                          ... 
You Broke Me First Tate McRae              0.0
You Right Doja Cat & The Weeknd            0.0
drivers license Olivia Rodrigo             0.5
rockstar (feat. 21 Savage) Post Malone     1.0
telepatía Kali Uchis                       0.0
Name: repeated_plays, Length: 90, dtype: float64

In [10]:
data['encoded_name'] = data['name'].map(target_mean)
data = data.drop(columns='name')

In [11]:
le = LabelEncoder()
data['encoded_id'] = le.fit_transform(data['id'])

In [12]:
scaler = StandardScaler()
data[['duration', 'energy','key','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','danceability','mode']] = scaler.fit_transform(data[['duration', 'energy','key','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','danceability','mode']])

In [13]:
# Step 1: Convert Timestamp to Datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

data['hour'] = data['timestamp'].dt.hour
data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)
data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)
#Drop the Original Timestamp
data = data.drop(columns=['timestamp'])

In [14]:
data = data.drop(columns=['id'])
data.head()

,duration,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,danceability,repeated_plays,encoded_name,encoded_id,hour,hour_sin,hour_cos
0,-0.475220,0.443793,1.061459,0.622267,0.876275,1.153077,0.069369,-0.315881,-0.694801,0.729530,1.554392,-0.661921,1,1.0,47,8,0.866025,-0.500000
1,-1.199018,-0.505709,0.796763,-1.926872,0.876275,-0.244177,0.226004,3.596394,-0.470974,-0.259219,1.662127,-0.653894,0,0.0,60,8,0.866025,-0.500000
2,-0.032300,1.411323,0.267370,1.136513,-1.141195,-0.226546,-1.091762,-0.315881,-0.850855,1.688701,-0.649632,0.453794,1,1.0,62,9,0.707107,-0.707107
3,-0.118723,0.636097,-1.320808,0.157113,0.876275,0.638840,0.019531,-0.315881,2.293421,-0.216965,-1.095840,0.253126,0,0.0,51,10,0.500000,-0.866025
4,-1.199018,-0.523738,0.796763,-0.065440,-1.141195,2.122779,-0.080147,-0.315881,2.159660,0.822489,1.962896,-0.421119,1,1.0,16,11,0.258819,-0.965926


In [15]:
from sklearn.model_selection import train_test_split

# Define features (X) and target variable (y)
X = data.drop(columns=['repeated_plays'])  # All columns except 'mode'
y = data['repeated_plays']  # Target variable

# Split into training (60%) and testing (40%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

In [16]:
from sklearn.linear_model import LogisticRegression

# Initialize and train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [17]:
from sklearn.metrics import accuracy_score, classification_report

# Predict on test data
y_pred = model.predict(X_test)

# Accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Classification report
print(classification_report(y_test, y_pred))

Accuracy: 0.9500
              precision    recall  f1-score   support

           0       0.91      1.00      0.95        21
           1       1.00      0.89      0.94        19

    accuracy                           0.95        40
   macro avg       0.96      0.95      0.95        40
weighted avg       0.95      0.95      0.95        40



In [18]:
feature_weights = dict(zip(X_train.columns, model.coef_[0]))

# Sort by absolute weight (importance)
sorted_features = sorted(feature_weights.items(), key=lambda x: abs(x[1]), reverse=True)

# Print feature importance
for feature, weight in sorted_features:
    print(f"{feature}: {weight:.4f}")

encoded_name: 3.4213
speechiness: 0.4711
energy: -0.3007
danceability: -0.2947
instrumentalness: -0.2440
key: -0.1431
loudness: 0.1419
mode: 0.1246
duration: -0.0755
valence: 0.0605
liveness: -0.0266
hour: -0.0213
tempo: -0.0212
encoded_id: 0.0103
hour_sin: 0.0090
acousticness: -0.0060
hour_cos: 0.0008
